In [3]:
import pandas as pd
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('week7').getOrCreate()

/home/lplab/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [6]:
data_wo_header = spark.read.option('inferSchema', True).option('header', False).csv('kddcup.data_10_percent_corrected')

In [7]:
column_names = data_wo_header.columns

In [9]:
column_names = [ "duration", "protocol_type", "service", "flag",
"src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
"hot", "num_failed_logins", "logged_in", "num_compromised",
"root_shell", "su_attempted", "num_root", "num_file_creations",
"num_shells", "num_access_files", "num_outbound_cmds",
"is_host_login", "is_guest_login", "count", "srv_count",
"serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
"same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
"dst_host_count", "dst_host_srv_count",
"dst_host_same_srv_rate", "dst_host_diff_srv_rate",
"dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
"dst_host_serror_rate", "dst_host_srv_serror_rate",
"dst_host_rerror_rate", "dst_host_srv_rerror_rate",
"label"]

data = data_wo_header.toDF(*column_names)

In [10]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

# Assuming 'data' is your DataFrame
# Handle missing values (example: drop rows with any null values)
data_cleaned = data.na.drop()

# Scale numerical features
input_numeric_only = data_cleaned.drop("protocol_type", "service", "flag")
assembler = VectorAssembler().setInputCols(input_numeric_only.columns).setOutputCol("featureVector")
scaler = StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector").setWithStd(True).setWithMean(False)

# Create a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages([assembler, scaler])
pipeline_model = pipeline.fit(data_cleaned)
scaled_data = pipeline_model.transform(data_cleaned)

IllegalArgumentException: Data type string of column label is not supported.